<a href="https://colab.research.google.com/github/uanushkatkd/CS6910_Assignment_3/blob/main/DL3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw' -O dataset.zip && unzip dataset.zip


--2023-05-03 13:58:37--  https://drive.google.com/uc?export=download&id=1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw
Resolving drive.google.com (drive.google.com)... 173.194.203.139, 173.194.203.113, 173.194.203.101, ...
Connecting to drive.google.com (drive.google.com)|173.194.203.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-1k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mf8t6uthf9tinolnmua53mrvgasueh6m/1683122250000/15657800450702375309/*/1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw?e=download&uuid=6322abff-1698-4a18-9f54-781f054654b3 [following]
--2023-05-03 13:58:44--  https://doc-04-1k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mf8t6uthf9tinolnmua53mrvgasueh6m/1683122250000/15657800450702375309/*/1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw?e=download&uuid=6322abff-1698-4a18-9f54-781f054654b3
Resolving doc-04-1k-docs.googleusercontent.com (doc-04-1k-docs.googleusercontent.com)... 74.125.142.132, 2

In [ ]:
import torch
import torch.nn as nn
print(torch.device('cuda:0'))
print(torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0
2.0.0+cu118
cuda


In [ ]:
# imports
import torch.nn as nn
import torch.optim as optim

import pandas as pd
import math
import torch
import torchvision
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import (
    DataLoader, random_split
)  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

from torchvision.datasets import ImageFolder
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pathlib


In [ ]:
def seed_everything(seed=1):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything()

In [ ]:
'''# Dataset load and Preprocessing
def data_preprocess(path):
   df = pd.read_csv(path, names=["English", "Translated"], sep=",").astype(str)
   df.dropna()
   english_texts = []
   target_texts = []
   for index, row in df.iterrows():
        ip_text = row['English']
        op_text = row['Translated']
        if ip_text == '' or op_text == '':
            continue
        op_text = "\t" + op_text + "\n"
        ip_text = "\t" + ip_text + "\n"
        
        english_texts.append(ip_text)
        target_texts.append(op_text)
   MAX_LEN_input = max([len(txt) for txt in english_texts])
   MAX_LEN_target = max([len(txt) for txt in target_texts])

   hindi_vocab = set()
   english_vocab = set()

   for word in target_texts:
     for char in word:
       hindi_vocab.add(char)

   for word in english_texts:
      for char in word:
        english_vocab.add(char)

   hindi_list = sorted(list(hindi_vocab))
   english_list = sorted(list(english_vocab))


   encoder_tokens = len(english_list)
   decoder_tokens = len(hindi_list)

   print("###############################",decoder_tokens)
   print("###############################",encoder_tokens)


        # Dict for char to index
   input_token_index = dict([(char, i) for i, char in enumerate(english_list)])
   target_token_index = dict([(char, i) for i, char in enumerate(hindi_list)])
   target_token_index["UNK"] = decoder_tokens

   #print(input_token_index)

        # Dict for index to char
   inv_input_token_index = dict({(value,key) for key,value in input_token_index.items()})
   #print(inv_input_token_index)
   inv_target_token_index = dict({(value,key) for key,value in target_token_index.items()})
   decoder_tokens+=1
   return MAX_LEN_input,MAX_LEN_target,hindi_list,english_list,encoder_tokens,decoder_tokens,input_token_index,target_token_index

def get_data(path,train_path):
   df = pd.read_csv(path, names=["English", "Translated"], sep=",").astype(str)
   df.dropna()
   MAX_LEN_input,MAX_LEN_target,hindi_list,english_list,encoder_tokens,decoder_tokens,input_token_index,target_token_index=data_preprocess(train_path)
   print("##############",MAX_LEN_input,MAX_LEN_target)

   decoder_target_data = np.zeros((df.shape[0], MAX_LEN_target,decoder_tokens), dtype="float32")
   print(decoder_target_data.shape)
   for i,target_text in enumerate(df["Translated"]):
            #print("Ths is i",i,target_text)

            target_text = '\t'+target_text+'\n'  
            for t, char in enumerate(target_text):
              #print(t,char)
              if t > 0:
                decoder_target_data[i, t - 1, target_token_index[char if char in target_token_index else "UNK"]] = 1.0
            decoder_target_data[i, t:,target_token_index["\n"]] = 1.0
   print("done")
   return ([[input_token_index[letter] for letter in list('\t'+word+'\n')] for word in df["English"]]),\
   ([[target_token_index[letter if letter in target_token_index else "UNK"] for letter in list('\t'+word+'\n')] for word in df["Translated"]]),decoder_target_data
    

def load_data():
    
  train_path  ="/content/aksharantar_sampled/hin/hin_train.csv"
  val_path  ="/content/aksharantar_sampled/hin/hin_valid.csv"
  test_path  ="/content/aksharantar_sampled/hin/hin_test.csv"


  encoder_train,decoder_train,decoder_target_train =get_data(train_path,train_path)
  encoder_val,decoder_val,decoder_target_val =get_data(val_path,train_path)
  encoder_test,decoder_test,decoder_target_test =get_data(test_path,train_path)


  encoder_train = [torch.tensor(lst) for lst in encoder_train]
  decoder_train = [torch.tensor(lst) for lst in decoder_train]

  encoder_val = [torch.tensor(lst) for lst in encoder_val]
  decoder_val = [torch.tensor(lst) for lst in decoder_val]
  
  encoder_test = [torch.tensor(lst) for lst in encoder_test]
  decoder_test = [torch.tensor(lst) for lst in decoder_test]

  # Pad the list of tensors to make them the same length
  encoder_train = torch.nn.utils.rnn.pad_sequence(encoder_train, batch_first=True, padding_value=0)
  decoder_train = torch.nn.utils.rnn.pad_sequence(decoder_train, batch_first=True, padding_value=0)

  encoder_val = torch.nn.utils.rnn.pad_sequence(encoder_val, batch_first=True, padding_value=0)
  decoder_val = torch.nn.utils.rnn.pad_sequence(decoder_val, batch_first=True, padding_value=0)

  encoder_test = torch.nn.utils.rnn.pad_sequence(encoder_test, batch_first=True, padding_value=0)
  decoder_test = torch.nn.utils.rnn.pad_sequence(decoder_test, batch_first=True, padding_value=0)
  
  return  encoder_train,decoder_train,decoder_target_train,encoder_val,decoder_val,decoder_target_val,encoder_test,decoder_test,decoder_target_test

'''

'# Dataset load and Preprocessing\ndef data_preprocess(path):\n   df = pd.read_csv(path, names=["English", "Translated"], sep=",").astype(str)\n   df.dropna()\n   english_texts = []\n   target_texts = []\n   for index, row in df.iterrows():\n        ip_text = row[\'English\']\n        op_text = row[\'Translated\']\n        if ip_text == \'\' or op_text == \'\':\n            continue\n        op_text = "\t" + op_text + "\n"\n        ip_text = "\t" + ip_text + "\n"\n        \n        english_texts.append(ip_text)\n        target_texts.append(op_text)\n   MAX_LEN_input = max([len(txt) for txt in english_texts])\n   MAX_LEN_target = max([len(txt) for txt in target_texts])\n\n   hindi_vocab = set()\n   english_vocab = set()\n\n   for word in target_texts:\n     for char in word:\n       hindi_vocab.add(char)\n\n   for word in english_texts:\n      for char in word:\n        english_vocab.add(char)\n\n   hindi_list = sorted(list(hindi_vocab))\n   english_list = sorted(list(english_vocab))\n\

In [ ]:
'''encoder_train,decoder_train,decoder_target_train,encoder_val,decoder_val,decoder_target_val,encoder_test,decoder_test,decoder_target_test= load_data()

print("##################################################################################################")
print("encoder_input",encoder_train.shape)
print("encoder_input",encoder_train[0])

print("decoder_target",decoder_train.shape)
print("encoder_input",decoder_train[0])


print("decoder_target_train",decoder_target_train[0])

'''

'encoder_train,decoder_train,decoder_target_train,encoder_val,decoder_val,decoder_target_val,encoder_test,decoder_test,decoder_target_test= load_data()\n\nprint("##################################################################################################")\nprint("encoder_input",encoder_train.shape)\nprint("encoder_input",encoder_train[0])\n\nprint("decoder_target",decoder_train.shape)\nprint("encoder_input",decoder_train[0])\n\n\nprint("decoder_target_train",decoder_target_train[0])\n\n'

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
class TransliterationDataset(Dataset):
    def __init__(self, file_path, src_lang, trg_lang):
        self.translations = pd.read_csv(file_path, header=None, names=[src_lang, trg_lang])
        self.src_lang = src_lang
        self.trg_lang = trg_lang
        self.src_vocab = {char: i+2 for i, char in enumerate(sorted(list(set(''.join(self.translations[src_lang].tolist())))))}
        print(self.src_vocab)
        self.src_vocab['<pad>'] = 0
        self.src_vocab['<unk>'] = 1
        self.trg_vocab = {char: i+2 for i, char in enumerate(sorted(list(set(''.join(self.translations[trg_lang].tolist())))))}
        


        # Extract the unique characters in the source and target languages
        src_chars = sorted(set(''.join(self.translations[src_lang])))
        trg_chars = sorted(set(''.join(self.translations[trg_lang])))

        # Assign an index to each character in the source and target languages
        self.char_to_idx = {char: idx for idx, char in enumerate(trg_chars)}
        self.idx_to_char = {idx: char for char, idx in self.char_to_idx.items()}

        
        self.trg_vocab['<pad>'] = 0
        self.trg_vocab['<unk>'] = 1
        self.max_src_len = max([len(word) for word in self.translations[src_lang].tolist()])
        print(self.max_src_len)
        self.max_trg_len = max([len(word) for word in self.translations[trg_lang].tolist()])
        print("trg vocab",len(self.trg_vocab))
        print("src vocab",len(self.src_vocab))


    def __len__(self):
        return len(self.translations)

    def target_to_one_hot(self,target_word, char_to_idx):
        num_trg_chars = len(char_to_idx)
        max_target_len = self.max_trg_len

        # Create a tensor of zeros for the one-hot encoding
        one_hot = torch.zeros((max_target_len, num_trg_chars))

        # Encode each character in the target word as a one-hot vector
        for i, char in enumerate(target_word):
            char_idx = char_to_idx[char]
            one_hot[i][char_idx] = 1

        return one_hot
    def __getitem__(self, idx):
        src_word = self.translations.iloc[idx][self.src_lang]
        trg_word = self.translations.iloc[idx][self.trg_lang]
        #print(src_word)
        #print(trg_word)

        src = [self.src_vocab.get(char, self.src_vocab['<unk>']) for char in src_word]
        trg = [self.trg_vocab.get(char, self.trg_vocab['<unk>']) for char in trg_word]

        src_len = len(src)
        trg_len = len(trg)

        src_pad = [self.src_vocab['<pad>']] * (self.max_src_len - src_len)
        trg_pad = [self.trg_vocab['<pad>']] * (self.max_trg_len - trg_len)

        src.extend(src_pad)
        trg.extend(trg_pad)

        src = torch.LongTensor(src)
        trg = torch.LongTensor(trg)
        trg_one_hot = self.target_to_one_hot(trg_word,self.char_to_idx)
        #print(trg_one_hot.shape)

        return src, trg, src_len, trg_len,trg_one_hot





In [ ]:
# Print the source word, target word, and source length

train_path  ="/content/aksharantar_sampled/hin/hin_train.csv"
val_path  ="/content/aksharantar_sampled/hin/hin_valid.csv"
test_path  ="/content/aksharantar_sampled/hin/hin_test.csv"

train_dataset = TransliterationDataset('/content/aksharantar_sampled/hin/hin_train.csv', 'English', 'Devanagari')
val_dataset = TransliterationDataset('/content/aksharantar_sampled/hin/hin_valid.csv', 'English', 'Devanagari')
test_dataset = TransliterationDataset('/content/aksharantar_sampled/hin/hin_test.csv', 'English', 'Devanagari')



train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for batch_idx, (src, trg, src_len, trg_len, trg_one_hot) in enumerate(test_loader):
    
    print('Source word:',src.shape)
    print('Target word:', trg.shape)
    print('Target one hot word:', trg_one_hot.shape)

    #print('Source length:', src_len)
    print('Source length:', trg_len)
    if batch_idx==0:
      break

#print("decoder_target_train",decoder_target_train[1::])

{'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27}
24
trg vocab 66
src vocab 28
{'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27}
22
trg vocab 63
src vocab 28
{'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27}
26
trg vocab 65
src vocab 28
Source word: torch.Size([64, 26])
Target word: torch.Size([64, 20])
Target one hot word: torch.Size([64, 20, 63])
Source length: tensor([ 7,  7,  4,  8, 11, 11,  9,  5,  9,  4,  5,  3, 11,  5,  7,  5,  6,  6,
        10,  8,  8,  7,

In [ ]:
# Model

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, cell_type):
        super(Encoder, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.cell_type = cell_type
        
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        
        if cell_type == 'rnn':
            self.rnn = nn.RNN(hidden_dim, hidden_dim, num_layers)
        elif cell_type == 'lstm':
            self.rnn = nn.LSTM(hidden_dim, hidden_dim, num_layers)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(hidden_dim, hidden_dim, num_layers)
        else:
            raise ValueError("Invalid cell type. Choose 'rnn', 'lstm', or 'gru'.")
        
    def forward(self, src):
        embedded = self.embedding(src)
        output, hidden = self.rnn(embedded)
        return output, hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, num_layers, cell_type):
        super(Decoder, self).__init__()
        
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.cell_type = cell_type
        
        self.embedding = nn.Embedding(output_dim, hidden_dim)
        
        if cell_type == 'rnn':
            self.rnn = nn.RNN(hidden_dim, hidden_dim, num_layers)
        elif cell_type == 'lstm':
            self.rnn = nn.LSTM(hidden_dim, hidden_dim, num_layers)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(hidden_dim, hidden_dim, num_layers)
        else:
            raise ValueError("Invalid cell type. Choose 'rnn', 'lstm', or 'gru'.")
        
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, input, hidden):
        input = input.unsqueeze(0)
        embedded = self.embedding(input)
        output, hidden = self.rnn(embedded, hidden)
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(device)
        
        encoder_output, encoder_hidden = self.encoder(src)
        
        decoder_hidden = encoder_hidden
        
        decoder_input = trg[0,:]
        
        for t in range(1, max_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[t] = decoder_output
            teacher_force = torch.rand(1) < teacher_forcing_ratio
            top1 = decoder_output.argmax(1)
            decoder_input = trg[t] if teacher_force else top1
        
        return outputs


In [ ]:
# Define hyperparameters
INPUT_DIM = 28
OUTPUT_DIM = 66
HIDDEN_DIM = 256
NUM_LAYERS = 2
CELL_TYPE = 'gru'
BATCH_SIZE = 64
LEARNING_RATE = 0.01
TEACHER_FORCING_RATIO = 0.5
EPOCHS = 10
trg_pad_idx=0

# Instantiate the Encoder and Decoder models
encoder = Encoder(INPUT_DIM, HIDDEN_DIM, NUM_LAYERS, CELL_TYPE).to(device)
decoder = Decoder(OUTPUT_DIM, HIDDEN_DIM, NUM_LAYERS, CELL_TYPE).to(device)

# Instantiate the Seq2Seq model with the Encoder and Decoder models
model = Seq2Seq(encoder, decoder).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train the model
for epoch in range(EPOCHS):
    epoch_loss = 0
    for batch_idx, (src, trg, src_len, trg_len, trg_one_hot) in enumerate(train_loader):
        #print(batch_idx)
        src = src.permute(1, 0)  # swapping the dimensions of src tensor
        trg = trg.permute(1, 0)  # swapping the dimensions of trg tensor

        src = src.to(device)
        trg = trg.to(device)
        
        optimizer.zero_grad()
        
        output = model(src, trg, TEACHER_FORCING_RATIO)
        
        # Ignore the first element of the output, which is initialized as all zeros
        # since we use it to store the output for the start-of-sequence token
        #print(output.shape[2])
        
        output = output[1:].reshape(-1, output.shape[2])
        #print(output.shape)
        #print(trg.shape)
        trg = trg[1:].reshape(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        if batch_idx % 1000 == 0:
            print(f"Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item()}")

Epoch: 0, Batch: 0, Loss: 4.210180282592773
Epoch: 1, Batch: 0, Loss: 1.6010288000106812
Epoch: 2, Batch: 0, Loss: 1.6040430068969727
Epoch: 3, Batch: 0, Loss: 1.6406605243682861
Epoch: 4, Batch: 0, Loss: 1.5708993673324585
Epoch: 5, Batch: 0, Loss: 1.7897568941116333
Epoch: 6, Batch: 0, Loss: 1.7383331060409546
Epoch: 7, Batch: 0, Loss: 1.5816158056259155
Epoch: 8, Batch: 0, Loss: 1.3727948665618896
Epoch: 9, Batch: 0, Loss: 1.5963302850723267


In [ ]:
#Training and check accuracy function



# Define hyperparameters
INPUT_DIM = 28
OUTPUT_DIM = 66
HIDDEN_DIM = 256
NUM_LAYERS = 2
CELL_TYPE = 'gru'
BATCH_SIZE = 64
LEARNING_RATE = 0.01
TEACHER_FORCING_RATIO = 0.5
EPOCHS = 10
trg_pad_idx=0

# Instantiate the Encoder and Decoder models
encoder = Encoder(INPUT_DIM, HIDDEN_DIM, NUM_LAYERS, CELL_TYPE).to(device)
decoder = Decoder(OUTPUT_DIM, HIDDEN_DIM, NUM_LAYERS, CELL_TYPE).to(device)

# Instantiate the Seq2Seq model with the Encoder and Decoder models
model = Seq2Seq(encoder, decoder).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train the model
for epoch in range(EPOCHS):
    epoch_loss = 0
    for batch_idx, (src, trg, src_len, trg_len, trg_one_hot) in enumerate(train_loader):
        #print(batch_idx)
        src = src.permute(1, 0)  # swapping the dimensions of src tensor
        trg = trg.permute(1, 0)  # swapping the dimensions of trg tensor

        src = src.to(device)
        trg = trg.to(device)
        
        optimizer.zero_grad()
        
        output = model(src, trg, TEACHER_FORCING_RATIO)
        
        # Ignore the first element of the output, which is initialized as all zeros
        # since we use it to store the output for the start-of-sequence token
        #print(output.shape[2])
        
        output = output[1:].reshape(-1, output.shape[2])
        #print(output.shape)
        #print(trg.shape)
        trg = trg[1:].reshape(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        if batch_idx % 1000 == 0:
            print(f"Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item()}")

Epoch: 0, Batch: 0, Loss: 4.1905717849731445
Epoch: 1, Batch: 0, Loss: 1.9107811450958252
Epoch: 2, Batch: 0, Loss: 1.8196845054626465
Epoch: 3, Batch: 0, Loss: 1.6872750520706177
Epoch: 4, Batch: 0, Loss: 1.5151541233062744
Epoch: 5, Batch: 0, Loss: 1.4219584465026855
Epoch: 6, Batch: 0, Loss: 1.528254747390747
Epoch: 7, Batch: 0, Loss: 1.746031403541565
Epoch: 8, Batch: 0, Loss: 1.6486035585403442
Epoch: 9, Batch: 0, Loss: 1.6557128429412842


In [ ]:
# final train